In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# os.environ['XLA_FLAGS'] ='--xla_gpu_deterministic_ops=true'


os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".5"

import torch
torch.multiprocessing.set_start_method('spawn')

In [3]:
# Add parent folder to path (to run this file from subdirectories)
(parent_folder_path, current_dir) = os.path.split(os.path.abspath(''))
sys.path.append(parent_folder_path)

# add git submodule to path to allow imports to work
submodule_name = 'AlphaTrade'
sys.path.append(os.path.join(parent_folder_path, submodule_name))

print(sys.path)

from gymnax_exchange.jaxob.jorderbook import OrderBook
import gymnax_exchange.jaxob.JaxOrderBookArrays as job

['/data1/sascha/LOBS5/lob', '/data1/sascha/AlphaTrade', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python310.zip', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/lib-dynload', '', '/home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages', '/data1/sascha/LOBS5', '/data1/sascha/LOBS5/AlphaTrade']


In [4]:
# from argparse import Namespace
from glob import glob
import numpy as onp
import pandas as pd
# from functools import partial
# from typing import Union, Optional
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
# from line_profiler import LineProfiler

import jax
import jax.numpy as jnp
from jax.nn import one_hot
# from jax import random
# from jax.scipy.linalg import block_diag
# from flax import jax_utils
# from flax.training import checkpoints
# import orbax

#from lob.lob_seq_model import BatchLobPredModel
# from lob.train_helpers import create_train_state, eval_step, prep_batch, cross_entropy_loss, compute_accuracy
from s5.ssm import *
# from s5.ssm_init import make_DPLR_HiPPO
# from s5.dataloading import make_data_loader
# from lob_seq_model import LobPredModel
from encoding import Vocab, Message_Tokenizer
# from lobster_dataloader import LOBSTER_Dataset, LOBSTER_Subset, LOBSTER_Sampler, LOBSTER

import preproc
# import inference
from lob import inference_no_errcorr as inference
import validation_helpers as valh
from lob.init_train import init_train_state, load_checkpoint, load_metadata, load_args_from_checkpoint
# import lob.encoding as encoding

# necessary for flax checkpoints to be loaded in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

from functools import partial



CUDA backend failed to initialize: jaxlib/cuda/versions_helpers.cc:99: operation cuInit(0) failed: CUDA_ERROR_NO_DEVICE.(Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
2024-07-30 20:50:55.158902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 20:50:55.179966: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 20:50:55.186147: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 20:50:56.011449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Forecasting

In [5]:
import lob.evaluation as eval
from preproc import transform_L2_state

In [18]:
n_gen_msgs = 100  #500 # how many messages to generate into the future
n_messages_conditional = 500
n_eval_messages = 100  # how many to load from dataset 
eval_seq_len = (n_eval_messages-1) * Message_Tokenizer.MSG_LEN

data_levels = 10
# TODO: deprecated - remove from functions
sim_book_levels = 20 # 10  # order book simulator levels
sim_queue_len = 100  # per price in sim, how many orders in queue

n_vol_series = 500  # how many book volume series model uses as input

v = Vocab()
n_classes = len(v)
book_dim = 501 #b_enc.shape[1]
eval_book_seq_len = eval_seq_len


rng = jax.random.key(42)
rng, rng_ = jax.random.split(rng)
sample_top_n = 50
tick_size = 100

In [7]:
# entire test set after training data
stock='GOOG'


if stock == 'GOOG':
    data_dir = '/data1/sascha/data/GOOG2017to2019'
    ckpt_path='/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5'
elif stock == 'INTC':
    raise NotImplementedError("Nothing trained for INTC yet")
elif stock == 'TSLA':
    raise Warning("Saved Model was trained on GOOGLE data. Generating for TSLA")
    data_dir = '/data1/sascha/data/lobster_proc'
    ckpt_path = '/data1/sascha/data/checkpoints/honest-oath-159_3kn3xbd5' # Dummy model trained on just 5 days... for debugging. 



In [21]:
args = load_metadata(ckpt_path)
ds = inference.get_dataset(data_dir, n_messages_conditional, n_eval_messages)

new_train_state, model_cls = init_train_state(
    args,
    n_classes=n_classes,
    seq_len=eval_seq_len,
    book_dim=book_dim,
    book_seq_len=eval_book_seq_len,
)



# ckpt = load_checkpoint(
#     new_train_state,
#     ckpt_path,
#     train=False,
# )
# state = ckpt['model']


import chex
chex.clear_trace_counter()

Dummy input shapes (msg,book) ((Array(8, dtype=int32, weak_type=True), Array(2178, dtype=int32, weak_type=True)), 
 (Array(8, dtype=int32, weak_type=True), Array(2178, dtype=int32, weak_type=True), Array(501, dtype=int32, weak_type=True)))
(12012, 512)
configuring standard optimization setup
[*] Trainable Parameters: 25699641
(12012, 512)
(1, 12012, 512)


In [17]:
len(ds.message_files)

743

In [16]:
import logging
# logging.basicConfig(filename='ar_debug.log', level=logging.DEBUG)
fhandler = logging.FileHandler(filename='generation_debug.log', mode='w')
logger = logging.getLogger()
if (logger.hasHandlers()):
    logger.handlers.clear()
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)
# logger.setLevel(logging.WARNING)

In [19]:
model = model_cls(training=False, step_rescale=1.0)


In [22]:
i = 200
l2_state_n = 20
m_seq, _, b_seq_pv, msg_seq_raw, book_l2_init = ds[int(i)]

b_seq = jnp.array(transform_L2_state(b_seq_pv, n_vol_series, 100))
m_seq_raw_inp = msg_seq_raw[: n_messages_conditional]

# with jax.disable_jit():

# initialise simulator
sim_init, sim_state_init = inference.get_sim(
    book_l2_init,  # book state before any messages
    m_seq_raw_inp, # messages to replay to init sim
)
# # book state after initialisation (replayed messages)
# l2_book_state_init = sim_init.get_L2_state(sim_state_init, l2_state_n)

m_seq, b_seq, msgs_decoded, l2_book_states, num_errors = inference.generate(
    sim_init,
    new_train_state,
    model,
    args.batchnorm,
    v.ENCODING,
    sample_top_n,
    tick_size,
    m_seq[:11000],
    b_seq[:500],
    n_gen_msgs,
    sim_state_init,
    rng,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 # # book state after initialisation (replayed messages)                                     │
│   16 # l2_book_state_init = sim_init.get_L2_state(sim_state_init, l2_state_n)                    │
│   17                                                                                             │
│ ❱ 18 m_seq, b_seq, msgs_decoded, l2_book_states, num_errors = inference.generate(                │
│   19 │   sim_init,                                                                               │
│   20 │   new_train_state,                                                                        │
│   21 │   model,                                                                                  │
│                                                                                                  │
│ /home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages/jax/_src/traceback_util │
│ .py:179 in reraise_with_filtered_traceback                                                       │
│                                                                                                  │
│   176   def reraise_with_filtered_traceback(*args, **kwargs):                                    │
│   177 │   __tracebackhide__ = True                                                               │
│   178 │   try:                                                                                   │
│ ❱ 179 │     return fun(*args, **kwargs)                                                          │
│   180 │   except Exception as e:                                                                 │
│   181 │     mode = _filtering_mode()                                                             │
│   182 │     if _is_under_reraiser(e) or mode == "off":                                           │
│                                                                                                  │
│ /home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages/jax/_src/pjit.py:305 in │
│ cache_miss                                                                                       │
│                                                                                                  │
│    302                                                                                           │
│    303   @api_boundary                                                                           │
│    304   def cache_miss(*args, **kwargs):                                                        │
│ ❱  305 │   outs, out_flat, out_tree, args_flat, jaxpr, attrs_tracked = _python_pjit_helper(      │
│    306 │   │   jit_info, *args, **kwargs)                                                        │
│    307 │   executable = _read_most_recent_pjit_call_executable(jaxpr)                            │
│    308 │   maybe_fastpath_data = _get_fastpath_data(                                             │
│                                                                                                  │
│ /home/sascha/miniconda3/envs/alphatrade_new/lib/python3.10/site-packages/jax/_src/pjit.py:172 in │
│ _python_pjit_helper                                                                              │
│                                                                                                  │
│    169                                                                                           │
│    170 def _python_pjit_helper(jit_info, *args, **kwargs):                                       │
│    171   (args_flat, _, params, _, out_tree, _, arg_names,                                       │
│ ❱  172    attrs_tracked) = _infer_params(jit_info, args, kw